In [1]:
from matplotlib import pyplot as plt
import numpy as np
np.random.seed(42)

In [13]:
from data_processing import read_edinburgh_data 
# set sampling rate as 9 so that the velocities in the df are correct
edinburgh_data, edinburgh_agent_ids = read_edinburgh_data(sampling_rate=9)

/Users/kriis/miniforge3/envs/tensorflow_m1/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


reading:./OpenTraj/datasets/Edinburgh/annotations/tracks.01Sep.txt


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 2342/2342 [01:03<00:00, 36.79it/s]


Warning! too big dt in [Edinburgh]
finish


In [2]:
from data_processing import read_eth_data
eth_data, eth_agent_ids = read_eth_data()

### Neural network predictions

In [6]:
from keras import models
from keras import backend as K

def RMSE(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true))) 

model = models.load_model('models/Edinburgh_NN_5_point_pred_velocity', custom_objects={"RMSE": RMSE})

In [10]:
edinburgh_test_agent_ids = np.load('test_agent_ids.npy')

In [22]:
def NN_prediction(data, agent_id):
    seen_x = np.array(data[data['agent_id'] == agent_id].iloc[:5]['pos_x'])
    seen_y = np.array(data[data['agent_id'] == agent_id].iloc[:5]['pos_y'])
    
    seen_vel_x = np.array(data[data['agent_id'] == agent_id].iloc[1:5]['vel_x'])
    seen_vel_y = np.array(data[data['agent_id'] == agent_id].iloc[1:5]['vel_y'])
    
    unseen_x = np.array(data[data['agent_id'] == agent_id].iloc[5:10]['pos_x'])
    unseen_y = np.array(data[data['agent_id'] == agent_id].iloc[5:10]['pos_y'])

    pred_vel_x, pred_vel_y = model.predict(np.array([np.column_stack((seen_vel_x, seen_vel_y))]))
    pred_vel_x = pred_vel_x[0] # destructure the prediction array as it is [[x1, x2, x3...]]
    pred_vel_y = pred_vel_y[0]
    
    pred_coord_x = [seen_x[-1] + pred_vel_x[0]]
    pred_coord_y = [seen_y[-1] + pred_vel_y[0]]
    
    for i in range(1, 5):
        pred_coord_x.append(pred_coord_x[-1] + pred_vel_x[i])
        pred_coord_y.append(pred_coord_y[-1] + pred_vel_y[i])

    FDE = np.sqrt((pred_coord_x[-1] - unseen_x[-1])**2 + (pred_coord_y[-1] - unseen_y[-1])**2)
    return pred_coord_x, pred_coord_y, FDE




### Baseline models

In [25]:
def generate_data(data, begin_idx, agent_id, num_steps=5):
    sample_x = data["pos_x"].loc[data["agent_id"]==agent_id][begin_idx:begin_idx+num_steps].values
    sample_y = data["pos_y"].loc[data["agent_id"]==agent_id][begin_idx:begin_idx+num_steps].values

    test_x = data["pos_x"].loc[data["agent_id"]==agent_id][begin_idx+num_steps:begin_idx+2*num_steps].values
    test_y = data["pos_y"].loc[data["agent_id"]==agent_id][begin_idx+num_steps:begin_idx+2*num_steps].values
    
    return sample_x, sample_y, test_x, test_y

In [35]:
def avg_vel_const(data, agent_id):
    sample_x, sample_y, test_x, test_y = generate_data(data, 0, agent_id, 5)
    
    sample_vel_x = [sample_x[i] - sample_x[i-1] for i in range(1, len(sample_x))]
    sample_vel_y = [sample_y[i] - sample_y[i-1] for i in range(1, len(sample_y))]
    
    avg_vel_x = np.mean(sample_vel_x)
    avg_vel_y = np.mean(sample_vel_y)
    
    pred_x = [sample_x[-1] + i*avg_vel_x for i in range(1, 6)]
    pred_y = [sample_y[-1] + i*avg_vel_y for i in range(1, 6)]
    
    FDE = np.sqrt((pred_x[-1] - test_x[-1])**2 + (pred_y[-1] - test_y[-1])**2)
    return FDE

def last_vel_const(data, agent_id):
    sample_x, sample_y, test_x, test_y = generate_data(data, 0, agent_id, 5)
    
    sample_vel_x = [sample_x[i] - sample_x[i-1] for i in range(1, len(sample_x))]
    sample_vel_y = [sample_y[i] - sample_y[i-1] for i in range(1, len(sample_y))]
    
    last_vel_x = sample_vel_x[-1]
    last_vel_y = sample_vel_y[-1]
    
    pred_x = [sample_x[-1] + i*last_vel_x for i in range(1, 6)]
    pred_y = [sample_y[-1] + i*last_vel_y for i in range(1, 6)]
    
    FDE = np.sqrt((pred_x[-1] - test_x[-1])**2 + (pred_y[-1] - test_y[-1])**2)
    return FDE

def discount_vel_const(data, agent_id):
    sample_x, sample_y, test_x, test_y = generate_data(data, 0, agent_id, 5)
    
    sample_vel_x = [sample_x[i] - sample_x[i-1] for i in range(1, len(sample_x))]
    sample_vel_y = [sample_y[i] - sample_y[i-1] for i in range(1, len(sample_y))]
    
    disc_avg_x = (0.8**3*sample_vel_x[0] + 0.8**2*sample_vel_x[1] + 0.8*sample_vel_x[2] + sample_vel_x[3]) / (0.8**3 + 0.8**2 + 0.8 + 1)
    disc_avg_y = (0.8**3*sample_vel_y[0] + 0.8**2*sample_vel_y[1] + 0.8*sample_vel_y[2] + sample_vel_y[3]) / (0.8**3 + 0.8**2 + 0.8 + 1)
    
    pred_x = [sample_x[-1] + i*disc_avg_x for i in range(1, 6)]
    pred_y = [sample_y[-1] + i*disc_avg_y for i in range(1, 6)]
    
    FDE = np.sqrt((pred_x[-1] - test_x[-1])**2 + (pred_y[-1] - test_y[-1])**2)
    return FDE

def avg_acc_const(data, agent_id):
    sample_x, sample_y, test_x, test_y = generate_data(data, 0, agent_id, 5)
    
    sample_vel_x = [sample_x[i] - sample_x[i-1] for i in range(1, len(sample_x))]
    sample_vel_y = [sample_y[i] - sample_y[i-1] for i in range(1, len(sample_y))]
    
    sample_acceleration_x = [sample_vel_x[i] - sample_vel_x[i-1] for i in range(1, len(sample_vel_x))]
    sample_acceleration_y = [sample_vel_y[i] - sample_vel_y[i-1] for i in range(1, len(sample_vel_y))]
    
    avg_acceleration_x = np.mean(sample_acceleration_x)
    avg_acceleration_y = np.mean(sample_acceleration_y)

    last_vel_x = sample_vel_x[-1]
    last_vel_y = sample_vel_y[-1]
    
    pred_x = [sample_x[-1] + last_vel_x + avg_acceleration_x]
    pred_y = [sample_y[-1] + last_vel_y + avg_acceleration_y]
    for i in range(1, 5):
        pred_x.append(pred_x[i-1] + last_vel_x + i * avg_acceleration_x)
        pred_y.append(pred_y[i-1] + last_vel_y + i * avg_acceleration_y)
        
    FDE = np.sqrt((pred_x[-1] - test_x[-1])**2 + (pred_y[-1] - test_y[-1])**2)
    return FDE
        
def last_acc_const(data, agent_id):
    sample_x, sample_y, test_x, test_y = generate_data(data, 0, agent_id, 5)
    
    sample_vel_x = [sample_x[i] - sample_x[i-1] for i in range(1, len(sample_x))]
    sample_vel_y = [sample_y[i] - sample_y[i-1] for i in range(1, len(sample_y))]
    
    avg_vel_x = np.mean(sample_vel_x)
    avg_vel_y = np.mean(sample_vel_y)
    
    sample_acceleration_x = [sample_vel_x[i] - sample_vel_x[i-1] for i in range(1, len(sample_vel_x))]
    sample_acceleration_y = [sample_vel_y[i] - sample_vel_y[i-1] for i in range(1, len(sample_vel_y))]
    
    last_acceleration_x = sample_acceleration_x[-1]
    last_acceleration_y = sample_acceleration_y[-1]
    
    last_vel_x = sample_vel_x[-1]
    last_vel_y = sample_vel_y[-1]
    
    pred_x = [sample_x[-1] + last_vel_x + last_acceleration_x]
    pred_y = [sample_y[-1] + last_vel_y + last_acceleration_y]
    for i in range(1, 5):
        pred_x.append(pred_x[i-1] + last_vel_x + i * last_acceleration_x)
        pred_y.append(pred_y[i-1] + last_vel_y + i * last_acceleration_y)
        
    FDE = np.sqrt((pred_x[-1] - test_x[-1])**2 + (pred_y[-1] - test_y[-1])**2)
    return FDE

def discount_acc_const(data, agent_id):
    sample_x, sample_y, test_x, test_y = generate_data(data, 0, agent_id, 5)
    
    sample_vel_x = [sample_x[i] - sample_x[i-1] for i in range(1, len(sample_x))]
    sample_vel_y = [sample_y[i] - sample_y[i-1] for i in range(1, len(sample_y))]
    
    avg_speed_x = np.mean(sample_vel_x)
    avg_speed_y = np.mean(sample_vel_y)
    
    sample_acc_x = [sample_vel_x[i] - sample_vel_x[i-1] for i in range(1, len(sample_vel_x))]
    sample_acc_y = [sample_vel_y[i] - sample_vel_y[i-1] for i in range(1, len(sample_vel_y))]
    
    discounted_avg_acc_x = (0.8**2*sample_acc_x[0] + 0.8*sample_acc_x[1] + sample_acc_x[2]) / (0.8**2 + 0.8 + 1)
    discounted_avg_acc_y = (0.8**2*sample_acc_y[0] + 0.8*sample_acc_y[1] + sample_acc_y[2]) / (0.8**2 + 0.8 + 1)

    last_vel_x = sample_vel_x[-1]
    last_vel_y = sample_vel_y[-1]
    
    pred_x = [sample_x[-1] + last_vel_x + discounted_avg_acc_x]
    pred_y = [sample_y[-1] + last_vel_y + discounted_avg_acc_y]
    
    for i in range(1, 5):
        pred_x.append(pred_x[i-1] + last_vel_x + i * discounted_avg_acc_x)
        pred_y.append(pred_y[i-1] + last_vel_y + i * discounted_avg_acc_y)


    FDE = np.sqrt((pred_x[-1] - test_x[-1])**2 + (pred_y[-1] - test_y[-1])**2)
    return FDE

### Evaluation

In [89]:
def run_models(data, test_agent_ids):
    NN_edinburgh_FDE = []
    avg_vel_const_FDE = []
    last_vel_const_FDE = []
    discount_vel_const_FDE = []
    avg_acc_const_FDE = []
    ast_acc_const_FDE = []
    discount_acc_const_FDE = []

    for idx, agent_id in enumerate(test_agent_ids):
        _, _, edinburgh_FDE = NN_prediction(data, agent_id)
        NN_edinburgh_FDE.append(edinburgh_FDE)
        avg_vel_const_FDE.append(avg_vel_const(edinburgh_data, agent_id))
        last_vel_const_FDE.append(last_vel_const(edinburgh_data, agent_id))
        discount_vel_const_FDE.append(discount_vel_const(edinburgh_data, agent_id))
        avg_acc_const_FDE.append(avg_acc_const(edinburgh_data, agent_id))
        last_acc_const_FDE.append(last_acc_const(edinburgh_data, agent_id))
        discount_acc_const_FDE.append(discount_acc_const(edinburgh_data, agent_id))
    
    res_FDE = [
        np.mean(NN_edinburgh_FDE),
        np.mean(avg_vel_const_FDE),
        np.mean(last_vel_const_FDE),
        np.mean(discount_vel_const_FDE),
        np.mean(avg_acc_const_FDE),
        np.mean(last_acc_const_FDE),
        np.mean(discount_acc_const_FDE)
    ]


In [91]:
index = [
    'Neural network',
    'Average velocity as const',
    'Last velocity as const',
    'Discounted velocity as const',
    'Average acceleration as const',
    'Last acceleration as const',
    'Discounted acceleration as const'
]

edinburgh_series = run_models(edinburgh_data, edinburgh_test_agent_ids)

eth_series = run_models(eth_data, eth_agent_ids)

d = {'Edinburgh' : pd.Series(edinburgh_series,
                       index = index),
      'ETH' : pd.Series(eth_series,
                        index = index)}

AttributeError: 'numpy.float64' object has no attribute 'append'

In [83]:
df = pd.DataFrame(d)

### Results

In [85]:
df

,Edinburgh,ETH
Neural network,2.209424,1.843934
Average velocity as const,2.472702,0.816345
Last velocity as const,2.140014,0.675217
Discounted velocity as const,2.318411,0.733045
Average acceleration as const,3.976010,1.179001
Last acceleration as const,4.992026,2.245425
Discounted acceleration as const,3.839182,1.175476
